# Selenium Python 웹 스크래핑 실습

## 0. 환경 설정
`pip install selenium webdriver-manager`

## 1. 기본 import 설정

In [2]:
# import the required library
from selenium import webdriver
 
# initialize an instance of the chrome driver (browser)
driver = webdriver.Chrome()

# visit your target site
driver.get("https://www.scrapingcourse.com/ecommerce/")

# output the full-page HTML
print(driver.page_source[:500])

# release the resources allocated by Selenium and shut down the browser
driver.quit()


<html lang="en-US"><head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="profile" href="https://gmpg.org/xfn/11">
<link rel="pingback" href="https://www.scrapingcourse.com/ecommerce/xmlrpc.php">
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-NZGD14H87G"></script>
<script>
	window.dataLayer = window.dataLayer || [];
	function gtag(){dataLayer.push(arguments);}
	gtag('js', new Date());
	gtag(


## 2. Headless Mode 설정

In [3]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")  # run in headless mode
driver = webdriver.Chrome(options=options)
# visit your target site
driver.get("https://www.scrapingcourse.com/ecommerce/")

# output the full-page HTML
print(driver.page_source[:500])

# release the resources allocated by Selenium and shut down the browser
driver.quit()

<html lang="en-US"><head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="profile" href="https://gmpg.org/xfn/11">
<link rel="pingback" href="https://www.scrapingcourse.com/ecommerce/xmlrpc.php">
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-NZGD14H87G"></script>
<script>
	window.dataLayer = window.dataLayer || [];
	function gtag(){dataLayer.push(arguments);}
	gtag('js', new Date());
	gtag(


## 3. 페이지의 특정 요소 추출하기

- find_element 함수를 통해서 스크래핑할 태그요소를 지정하여 가져올 수 있다.
- 일부 페이지는 로딩까지 시간이 걸릴 수 있으므로, `time.sleep`을 통해 모든 컨텐츠가 로드될 때까지 대기 후 스크래핑을 진행할 수 있다.
    - 대기하는 방법에는 다음과 같은 선택지가 있다.
    1. `time.sleep()`으로 지정하기
    2. `implicity_wait`으로 모든 컨텐츠가 로드될때까지 기다리기
    3. `WebDriverWait` 특정 조건 충족시까지 기다리기 
    실제론 3번이 제일 효과적이라고 한다.
- 태그요소는 id, 클래스 이름, css선택자, XPath 등 여러 방법으로 지정할 수 있다.

| 방법                                  | 특징              | 장점                              | 단점                                   |
| ----------------------------------- | --------------- | ------------------------------- | ------------------------------------ |
| **`time.sleep()`**                  | 고정 시간 대기        | 간단, 빠른 테스트용 적합                  | 비효율적 (불필요하게 오래 대기하거나 너무 짧아 오류 발생 가능) |
| **`implicitly_wait()`**             | 모든 요소 탐색에 공통 적용 | 코드 간결, 전역 적용 가능                 | 제어 부족, 요소 존재만 확인 (보임 여부 X)           |
| **`WebDriverWait` (Explicit Wait)** | 특정 조건 충족 시까지 대기 | 가장 유연하고 정확함 (예: 요소가 클릭 가능할 때까지) | 코드 작성 시 조건 명시 필요                     |

### 1. time.sleep() 활용

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random


options = webdriver.ChromeOptions()
options.add_argument("--headless=new")  # run in headless mode
driver = webdriver.Chrome(options=options,)

url = "https://m.sports.naver.com/index"

driver.get(url)
time.sleep(random.uniform(2, 3))  # 페이지 로딩 대기

# 기사 블록 요소 찾기
articles = driver.find_elements(By.CSS_SELECTOR, "div.mfc_tmplfeedmixed_template_body")


# 결과 저장용 리스트
extracted_articles = []

for article in articles:
    try:
        image = article.find_element(By.CSS_SELECTOR, "span.mfc_elemimagerectangle_image_box img").get_attribute("src")
        title = article.find_element(By.CSS_SELECTOR, ".mfc_comptextcard_text_title").text
        link = article.find_element(By.CSS_SELECTOR, "a.mfc_modmarginfixed_margin_thumb_link").get_attribute("href")
        channel = article.find_element(By.CSS_SELECTOR, ".mfc_compchannelsmall_channel_name").text

        article_data = {
            "title": title,
            "image": image,
            "url": link,
            "channel": channel,
        }

        extracted_articles.append(article_data)

    except Exception as e:
        print("오류 발생:", e)
        continue

# 출력
print(extracted_articles[:500])

# 드라이버 종료
driver.quit()


[{'title': 'LA다저스, 연장 13회 접전 끝에 7대5 승리... 김혜성은 대주자로 투입 후 타석에서 안타 기록', 'image': 'https://dthumb-phinf.pstatic.net/?src=%22http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA1MjRfMjE2%2FMDAxNzQ4MDg3MzE2NDA5.Hw3bVfR-rZqCMW_NZ7lUFdPwxgcXKk_FkXcVJULmxlwg.PPsf9VgZKhet1zLwcsq5Z9XekJWdwNk-eH4NwfzfgR8g.JPEG%2FIMG%EF%BC%BF2264.JPG%22&type=nf722_406&service=sports', 'url': 'https://blog.naver.com/indodgersss/223877142293', 'channel': '루크'}]


### 2. WebDriverWait활용
 

In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")  # run in headless mode
driver = webdriver.Chrome(options=options)

url = "https://m.sports.naver.com/index"
driver.get(url)

# 명시적 대기: 첫 기사 요소가 보일 때까지 최대 5초 대기
WebDriverWait(driver, 5).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "div.mfc_tmplfeedmixed_template_body"))
)

# 무한로딩을 방지하기 위한 조건 설정
max_scrolls = 15         # 최대 스크롤 횟수
stuck_limit = 3          # 콘텐츠 변화 없음 허용 횟수
max_articles = 100       # 최대 기사 수
scrolls = 0
stuck_count = 0

# 기사 저장 리스트 (초기화 위치 이동)
extracted_articles = []

# 스크롤 크기 구하기
scroll_height = driver.execute_script("return document.body.scrollHeight")
print("스크롤 높이:", scroll_height)

while scrolls < max_scrolls and stuck_count < stuck_limit and len(extracted_articles) < max_articles:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 명시적 대기 or 랜덤 sleep (둘 다 적절히 병행 가능)
    time.sleep(random.uniform(2, 3))

    # 스크롤 후 높이 확인
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_scroll_height == scroll_height:
        stuck_count += 1
    else:
        stuck_count = 0
        scroll_height = new_scroll_height

    scrolls += 1

    # 현재까지 로드된 기사들
    articles = driver.find_elements(By.CSS_SELECTOR, "div.mfc_tmplfeedmixed_template_body")
    
    for article in articles:
        try:
            image = article.find_element(By.CSS_SELECTOR, "span.mfc_elemimagerectangle_image_box img").get_attribute("src")
            title = article.find_element(By.CSS_SELECTOR, ".mfc_comptextcard_text_title").text
            link = article.find_element(By.CSS_SELECTOR, "a.mfc_modmarginfixed_margin_thumb_link").get_attribute("href")
            channel = article.find_element(By.CSS_SELECTOR, ".mfc_compchannelsmall_channel_name").text

            article_data = {
                "title": title,
                "image": image,
                "url": link,
                "channel": channel,
            }

            if article_data not in extracted_articles:
                extracted_articles.append(article_data)

        except Exception:
            continue

        if len(extracted_articles) >= max_articles:
            break

# 결과 출력
print(f"총 수집된 기사 수: {len(extracted_articles)}")
for article in extracted_articles:
    print(article)

# 드라이버 종료 
driver.quit()


스크롤 높이: 4829
총 수집된 기사 수: 100
{'title': '영화 아닙니까? 이강인 우승 메달 여자친구한테 걸어주기.. 남자다 남자..', 'image': 'https://dthumb-phinf.pstatic.net/?src=%22http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA1MjBfMTUw%2FMDAxNzQ3NjY4NzYxMDI2.NFNaYIzqcqkEzPhvtwAHLBmNVdzxOlRfpHo8rQF_73wg.FSmKI64GIBTe-RSNb2fYhs1OfP4Utl7tKtt5TeT2z-Ug.PNG%2F%EC%A0%9C%EB%AA%A9%EC%9D%84-%EC%9E%85%EB%A0%A5%ED%95%B4%EC%A3%BC%EC%84%B8%EC%9A%94_-001_-_2025-05-20T003228.690.png%22&type=nf722_406&service=sports', 'url': 'https://blog.naver.com/orina_/223871136773', 'channel': '오리나'}
{'title': '전세계 당구팬 감동시킨 조재호만 가능한 대인배 행동', 'image': 'https://dthumb-phinf.pstatic.net/?src=%22http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA1MjJfOTcg%2FMDAxNzQ3OTE3ODE2NjEy.R4MsuWoIhlBL9RywQfrT7NoJ6BhELwKAd_sEIu3EuZkg.yAj3JnMzoJ2Vw4JcX7cAFtMC79nJxVt9j3KhwiSZDwsg.PNG%2F%EC%A0%9C%EB%AA%A9%EC%9D%84-%EC%9E%85%EB%A0%A5%ED%95%B4%EC%A3%BC%EC%84%B8%EC%9A%94_-002_(1).png%22&type=nf722_406&service=sports', 'url': 'https://blog.naver.com/39messi/223874598340', 'channel': '당구메시'}
{'ti

## 4. 추출한 요소를 csv파일로 저장하기

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")  # run in headless mode
driver = webdriver.Chrome(options=options)

url = "https://m.sports.naver.com/index"

driver.get(url)
time.sleep(random.uniform(2, 3))  # 페이지 로딩 대기


# 기사 블록 요소 찾기
articles = driver.find_elements(By.CSS_SELECTOR, "div.mfc_tmplfeedmixed_template_body")


# 결과 저장용 리스트
extracted_articles = []

for article in articles:
    try:
        image = article.find_element(By.CSS_SELECTOR, "span.mfc_elemimagerectangle_image_box img").get_attribute("src")
        title = article.find_element(By.CSS_SELECTOR, ".mfc_comptextcard_text_title").text
        link = article.find_element(By.CSS_SELECTOR, "a.mfc_modmarginfixed_margin_thumb_link").get_attribute("href")
        channel = article.find_element(By.CSS_SELECTOR, ".mfc_compchannelsmall_channel_name").text

        article_data = {
            "title": title,
            "image": image,
            "url": link,
            "channel": channel,
        }

        extracted_articles.append(article_data)

    except Exception as e:
        print("오류 발생:", e)
        continue

# CSV 파일로 저장
csv_file = "articles.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["title", "image", "url", "channel"])
    writer.writeheader()
    for article in extracted_articles:
        writer.writerow(article)
        
print(f"저장 완료: {csv_file}") 
# 드라이버 종료
driver.quit()


저장 완료: articles.csv


## 5. 브라우저에서 다루듯 조정하는 법

### 1. 스크롤

일부 웹페이지에서는 스크롤을 해야 컨텐츠가 로드가 되는 JS기능이 적용되어있을 수 있다.
이에 스크롤을 구현하는 방법을 실습하고자 한다.

In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")  # run in headless mode
driver = webdriver.Chrome(options=options)
url = "https://m.sports.naver.com/index"
driver.get(url)
time.sleep(random.uniform(2, 3))  # 페이지 로딩 대기

# 무한로딩을 방지하기 위한 조건 설정
max_scrolls = 15         # 최대 스크롤 횟수
stuck_limit = 3          # 콘텐츠 변화 없음 허용 횟수
max_articles = 100       # 최대 기사 수
scrolls = 0
stuck_count = 0

# 스크롤 크기 구하기
scroll_height = driver.execute_script("return document.body.scrollHeight")
print("스크롤 높이:", scroll_height)

while scrolls < max_scrolls and stuck_count < stuck_limit and len(extracted_articles) < max_articles:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 3))

    # 스크롤 후 높이 확인
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_scroll_height == scroll_height:
        stuck_count += 1
    else:
        stuck_count = 0
        scroll_height = new_scroll_height

    scrolls += 1

    # 현재까지 로드된 기사들
    articles = driver.find_elements(By.CSS_SELECTOR, "div.mfc_tmplfeedmixed_template_body")
    
    for article in articles:
        try:
            image = article.find_element(By.CSS_SELECTOR, "span.mfc_elemimagerectangle_image_box img").get_attribute("src")
            title = article.find_element(By.CSS_SELECTOR, ".mfc_comptextcard_text_title").text
            link = article.find_element(By.CSS_SELECTOR, "a.mfc_modmarginfixed_margin_thumb_link").get_attribute("href")
            channel = article.find_element(By.CSS_SELECTOR, ".mfc_compchannelsmall_channel_name").text

            article_data = {
                "title": title,
                "image": image,
                "url": link,
                "channel": channel,
            }

            # 중복 방지
            if article_data not in extracted_articles:
                extracted_articles.append(article_data)

        except Exception as e:
            continue

        if len(extracted_articles) >= max_articles:
            break

    
# 결과 출력
print(f"총 수집된 기사 수: {len(extracted_articles)}")
for article in extracted_articles:
    print(article)
    
# 드라이버 종료 
driver.quit()


### 2. 스크린 샷 찍기

`driver.save_screenshot("스샷이름.png")`

### 3. 링크 클릭하기
`driver.find_element().click()`


### 4. 폼 구역 입력하기
`driver.find_element().send_keys("내용")`

### 5. JS 실행명령어 삽입하기
`driver.execute_script("명령어")`

### 6. 윈도우 크기 커스텀
반응형 웹의 수가 증가함에 따라 윈도우 크기를 조정하여 다른 환경의 화면을 확인할 필요도 있다. 이를 위한 명령어에는 두가지 방식이 있다.
-  `options.add_arguemnt("--window-size=<width>,<height>")`
-  `set_window_size(<width>, <height>)`